In [19]:
import os
import json
import pandas as pd
import traceback
import langchain
from dotenv import load_dotenv
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
api_key=os.getenv("OPENAI_API_KEY")

In [11]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4o-mini",temperature=.5)

In [13]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f7b82f4a6d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f7b82f58880>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key='sk-None-zfdG9I9cf9UvPgF93BE9T3BlbkFJsztDSHxVd7bDxW4Qr0yL', openai_proxy='')

In [22]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [24]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [26]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [27]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [28]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [29]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [30]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],output_variables=["quiz", "review"], verbose=True,)

In [35]:
file_path="/Users/itamarlevi/Desktop/MCQ_Generator/data.txt"

In [36]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [37]:
print(TEXT)


What is Retrieval-Augmented Generation?
Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

Why is Retrieval-Augmented Generation important?
LLMs are a key artificial intelligence (AI) technology powering intelligent chatbots and other natural language processing (NLP) applications. The goal is to create bots that can answer user quest

In [38]:
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [49]:

NUMBER=5 
SUBJECT="AI"
TONE="simple"

In [50]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:What is Retrieval-Augmented Generation?
Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

Why is Retrieval-Augmented Generation important?
LLMs are a key artificial intelligence (AI) technology powering intelligent chatbots and

In [51]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3726
Prompt Tokens:2928
Completion Tokens:798
Total Cost:0.0009179999999999999


In [52]:
response


{'text': 'What is Retrieval-Augmented Generation?\nRetrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization\'s internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.\n\nWhy is Retrieval-Augmented Generation important?\nLLMs are a key artificial intelligence (AI) technology powering intelligent chatbots and other natural language processing (NLP) applications. The goal is to create bots that can an

In [56]:
quiz_data = response.get("quiz")


In [57]:
quiz=json.loads(quiz_data)


In [58]:
quiz

{'1': {'mcq': 'What does Retrieval-Augmented Generation (RAG) aim to improve in large language models?',
  'options': {'a': 'The speed of response',
   'b': 'The accuracy and relevance of responses',
   'c': 'The size of the model',
   'd': 'The cost of training'},
  'correct': 'b'},
 '2': {'mcq': 'What is the main challenge faced by large language models (LLMs)?',
  'options': {'a': 'They are too small',
   'b': 'They always provide accurate information',
   'c': 'They may present false or outdated information',
   'd': 'They cannot generate original content'},
  'correct': 'c'},
 '3': {'mcq': 'How does RAG enhance user trust?',
  'options': {'a': 'By generating responses quickly',
   'b': 'By providing source citations and references',
   'c': 'By using only internal data',
   'd': 'By avoiding user queries'},
  'correct': 'b'},
 '4': {'mcq': 'What is the process of creating external data in RAG?',
  'options': {'a': 'Retraining the LLM',
   'b': 'Converting data into numerical repre

In [59]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [60]:
quiz_table_data


[{'MCQ': 'What does Retrieval-Augmented Generation (RAG) aim to improve in large language models?',
  'Choices': 'a: The speed of response | b: The accuracy and relevance of responses | c: The size of the model | d: The cost of training',
  'Correct': 'b'},
 {'MCQ': 'What is the main challenge faced by large language models (LLMs)?',
  'Choices': 'a: They are too small | b: They always provide accurate information | c: They may present false or outdated information | d: They cannot generate original content',
  'Correct': 'c'},
 {'MCQ': 'How does RAG enhance user trust?',
  'Choices': 'a: By generating responses quickly | b: By providing source citations and references | c: By using only internal data | d: By avoiding user queries',
  'Correct': 'b'},
 {'MCQ': 'What is the process of creating external data in RAG?',
  'Choices': 'a: Retraining the LLM | b: Converting data into numerical representations | c: Only using text data | d: Ignoring external sources',
  'Correct': 'b'},
 {'MCQ

In [61]:
quiz=pd.DataFrame(quiz_table_data)


In [64]:
quiz

,MCQ,Choices,Correct
0,What does Retrieval-Augmented Generation (RAG)...,a: The speed of response | b: The accuracy and...,b
1,What is the main challenge faced by large lang...,a: They are too small | b: They always provide...,c
2,How does RAG enhance user trust?,a: By generating responses quickly | b: By pro...,b
3,What is the process of creating external data ...,a: Retraining the LLM | b: Converting data int...,b
4,What happens after the relevant information is...,a: It is discarded | b: It is used to augment ...,b


In [62]:
quiz.to_csv("machinelearning.csv",index=False)


In [63]:

from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_02_2024_15_19_27'